In [2]:
classes = ['god', 'night', 'sunset', 'motorbike', 'clouds', 'baby', 'hair', 'icecream', 'shades', 'beach', 'sunrise']
print(len(classes))

11


In [141]:
import requests , time , hashlib , random
from itertools import cycle
from requests_html import HTMLSession
from bs4 import BeautifulSoup
# find_ip_address_url = "https://geoip-db.com/json/"

class Custom_Requests:
    use_proxy = None
    proxy_handler = None
    def __init__(self,use_proxy_param=False,use_session=True):
        if use_proxy_param:
            proxy_handler = PROXYHANDLER()
            self.proxy_handler = proxy_handler
        if use_session:
            self.session = HTMLSession()
        else:
            self.session = requests
            
    def get(self,url,parser="html.parser",request_type='get',headers=None
                     ,payload=None,params=None,try_times=20):
        retry_times = 0
        response = None
        while retry_times<=try_times:
            try:
                if self.proxy_handler:
                    proxy_option = self.proxy_handler.make_proxy()
                else:
                    proxy_option = None
                if request_type=='get':
                    response = self.session.get(url,proxies=proxy_option,headers=headers,
                                                data=payload,params=params)
                else:
                    response = self.session.post(url,proxies=proxy_option,headers=headers,
                                                data=payload,params=params)
                if response.ok:
                    break
            except:
                #traceback.print_exc()
                retry_times = retry_times + 1
        if response:
            if parser == 'json':
                response = response.json()
            elif parser == 'content':
                response = response.content
            elif parser == 'html.parser':
                response = BeautifulSoup(response.content,parser)
            elif parser == 'xml':
                response = BeautifulSoup(response.content,parser)
            else:
                response = response.content
        return response

class PROXYHANDLER():
    proxy_cycle = None
    proxies_list = []
    proxy_used = 0
    IP = None
    
    def __init__(self,recall_proxy_interval = 25,flag=False):
        self.flag = flag
        self.recall_proxy_interval = recall_proxy_interval
        try:    
            self.IP = self.get_ip_API()
        except:
            self.IP = self.get_ip()
        self.add_ip_to_whitelist(self.IP)
        self._call_for_tt_proxies()


    def _check_to_call_tt_proxy(self):
        if self.proxy_used % self.recall_proxy_interval == 0:
            self._call_for_tt_proxies()


    def _call_for_tt_proxies(self):
        try_times = 10
        while try_times != 0:
            try:
                self.obtain_proxy_list_from_tt_proxy()
                self.shuffel_proxies()
                break
            except:
                time.sleep(3)
                try_times = try_times - 1


    def shuffel_proxies(self,flag=False):
        random.shuffle(self.proxies_list)
        self.proxy_cycle = cycle(self.proxies_list)


    def make_proxy(self):
        PROXY = next(self.proxy_cycle)
        self.proxy_used = self.proxy_used + 1
        self._check_to_call_tt_proxy()
        return PROXY

    def get_proxy_list_from_tt_proxy(self):
        proxies = []
        secret = '50piVRZ9ZedEy2EfqXX9u3'
        params = {
            "license": "PBA5EE66C8FE5DE84",
            "time": int(time.time()),
            "cnt": 1000,
        }
        params["sign"] = hashlib.md5((params["license"] + str(params["time"]) + secret).encode('utf-8')).hexdigest()
        response = requests.get(
                    url="https://api.ttproxy.com/v1/obtain",
                    params=params,
                    headers={
                        "Content-Type": "text/plain; charset=utf-8",
                    },
                    data="1"
                ).json()
        proxies = response["data"]["proxies"]
        return proxies


    def obtain_proxy_list_from_tt_proxy(self,start=0,end=-1):
        proxies_list = self.get_proxy_list_from_tt_proxy()
        final_proxies = []
        for PROXY in proxies_list:
            if self.flag:
                pr = {"http"  : "http://" + PROXY}
            else:
                pr = {"http"  : "http://" + PROXY,"https" : "http://" + PROXY}
            final_proxies.append(pr)
        self.proxies_list = final_proxies


    def add_ip_to_whitelist(self,ip=''):
        secret = '50piVRZ9ZedEy2EfqXX9u3'
        params = {
        "license": "PBA5EE66C8FE5DE84",
        "time": int(time.time()),
        "cnt": 1000,
        }
        params["sign"] = hashlib.md5((params["license"] + str(params["time"]) + secret).encode('utf-8')).hexdigest()
        params['ip']=ip
        response = requests.get(
                url='https://api.ttproxy.com/v1/whitelist/add',
                params=params,
                headers={
                    "Content-Type": "text/plain; charset=utf-8",
                },
                data="1"
            ).json()
        return response

    def find_my_ip_address(self):
        ## importing socket module
        import socket
        ## getting the hostname by socket.gethostname() method
        hostname = socket.gethostname()
        ## getting the IP address using socket.gethostbyname() method
        ip_address = socket.gethostbyname(hostname)
        ## printing the hostname and ip_address
        # print(f"Hostname: {hostname}")
        # print(f"IP Address: {ip_address}")
        return ip_address

    def get_ip(self):
        import socket
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        try:
            # doesn't even have to be reachable
            s.connect(('10.255.255.255', 1))
            IP = s.getsockname()[0]
        except Exception:
            IP = '127.0.0.1'
        finally:
            s.close()
        return IP

    def get_ip_API(self):
        api_url = 'https://api.ipify.org?format=json'
        return requests.get(api_url).json()['ip']

    def get_ip_address_by_google(self):
        from requests_html import HTMLSession
        from bs4 import BeautifulSoup
        sess = HTMLSession()
        rrr = "https://www.google.com/search?ei=uihSYKirGpPB3LUP_Li3gAk&q=what+is+my+ip"
        res = sess.get(rrr).content
        res = BeautifulSoup(res,"lxml")
        res = res.find("span",style="font-size:20px").text
        return res
    
def get_images_from_hashtag_page(hashtag = "nightsky",scroll_page = 3):
    hashtag_url = f"https://www.instagram.com/explore/tags/{hashtag}/?__a=1"
    next_page_url = ""
    complete_url = hashtag_url+next_page_url
    goto_next_page = True
    image_urls_list = []
    while goto_next_page and scroll_page>0:
        complete_url = hashtag_url+next_page_url
        #response = requests.get(complete_url,headers=h).json()
        response = cr.get(complete_url,headers=h,parser='json')
#         response = requests.get(complete_url).json()
        page_info = response['graphql']['hashtag']["edge_hashtag_to_media"]["page_info"]
        if page_info["has_next_page"]:
            end_cursor = page_info["end_cursor"]
            complete_url = "&max_id={}".format(end_cursor)
        else:
            goto_next_page = False
        edges = response['graphql']['hashtag']["edge_hashtag_to_media"]["edges"]
        image_urls = [a["node"]["display_url"] for a in edges]
        image_urls_list = image_urls_list+image_urls
        scroll_page = scroll_page -1
        time.sleep(1)
    return list(set(image_urls_list))

In [135]:
cr = Custom_Requests(use_proxy_param=True, use_session=True)
cr.proxy_handler.IP

'49.37.74.245'

In [136]:
sessionid = '39673737047%3Acf6Ydg9lghnde6%3A23'
h = {'Accept-Language':'en-US,en;q=0.5',"Cookie":"sessionid={}".format(sessionid)}
h['x-csrftoken']="DkPugukmMfCunkG7pDS6dS0T6wnZQqBX"
h['content-type']='application/x-www-form-urlencoded'

In [137]:
# from tqdm.notebook import tqdm
# import requests
# import json
# from time import sleep
# image_links = {}
# for cls in tqdm(['fashion']):
#     max_id = ''
#     base_url = f"https://www.instagram.com/explore/tags/{cls}/?__a=1"
#     temp = []
#     while True:
#         sleep(2) # Be polite.

#         if max_id:
#             url = base_url + f"&max_id={max_id}"
#         else:
#             url = base_url
#         response = requests.get(url, headers = header)
#         response = json.loads(response.text)
#         #response = cr.get(url, parser = 'json', headers = header)
#         try:
#             max_id = response['graphql']['hashtag']['edge_hashtag_to_media']['page_info']['end_cursor']
#         except KeyError:
#             break
#         edges = response['graphql']['hashtag']["edge_hashtag_to_media"]["edges"]
#         image_urls = [a["node"]["display_url"] for a in edges]
#         temp += image_urls
#         print((len(set(temp))), end = '\r')
#         if (len(set(temp))) >= 1000:
#             break
#     image_links[cls] = temp

# import os
# import urllib.request
# i = 0
# for cls in tqdm(classes):
#     try:
#         os.mkdir(r'E:\insta/' + cls)
#     except:
#         pass
#     for url in image_links[cls]:
#         r = urllib.request.urlopen(url)
#         path = f'E:\insta/{cls}/{i}.png'
#         with open(path, "wb") as f:
#             f.write(r.read())
#             i += 1

In [142]:
igs = get_images_from_hashtag_page(hashtag = "toonapp",scroll_page = 3)
len(igs)

216

In [146]:
for a in igs[:10]:
    print(a)

https://instagram.faru4-1.fna.fbcdn.net/v/t51.2885-15/276019268_392437205578633_3670504318818026425_n.webp?stp=dst-jpg_e35&_nc_ht=instagram.faru4-1.fna.fbcdn.net&_nc_cat=101&_nc_ohc=Y7cvnwlmjR8AX-JwTUK&edm=ABZsPhsBAAAA&ccb=7-4&oh=00_AT-dFWrLYW2k-WBfUUd_LGZXIRpvmMdR72bcuym6wskoDg&oe=623F8F62&_nc_sid=4efc9f
https://instagram.fplu22-1.fna.fbcdn.net/v/t51.2885-15/276088810_1268930320262559_6013643365533612531_n.webp?stp=dst-jpg_e35&_nc_ht=instagram.fplu22-1.fna.fbcdn.net&_nc_cat=103&_nc_ohc=NYHHaGsTnCYAX9KurFc&edm=ABZsPhsBAAAA&ccb=7-4&oh=00_AT_CnTgVIcYjfmPVMdK0nDDngZxIRym1xJtSKoQUkYa3OA&oe=623F96EB&_nc_sid=4efc9f


In [145]:
len(set(igs))

216

In [139]:
igs.keys()

dict_keys(['graphql', 'showQRModal'])

In [21]:
igs['data'].keys()

dict_keys(['id', 'name', 'media_count', 'follow_status', 'following', 'allow_following', 'allow_muting_story', 'profile_pic_url', 'non_violating', 'related_tags', 'subtitle', 'social_context', 'social_context_profile_links', 'social_context_facepile_users', 'follow_button_text', 'show_follow_drop_down', 'formatted_media_count', 'challenge_id', 'destination_info', 'description', 'debug_info', 'fresh_topic_metadata', 'promo_banner', 'top', 'recent', 'content_advisory', 'warning_message'])

In [77]:
igs['data']['fresh_topic_metadata']

In [25]:
igs['data']['recent'].keys()


dict_keys(['sections', 'more_available', 'next_max_id', 'next_page', 'next_media_ids'])

In [80]:
igs['data']['recent']['next_max_id']

'QVFBVllBekJsYXFBdVJMMmluelJoTEJJQWlLY1ZJY2tLNDZQS3RtcE5GUHdPNWtqSnd0cnpjMDNPZmhGLU51UVluRHhCbndQWXcwYkl1SkhvdlRDNkw3Ng=='

In [60]:
igs['data']['top'].keys()

dict_keys(['sections', 'more_available', 'next_max_id', 'next_page', 'next_media_ids'])

In [30]:
igs['data']['recent']['next_max_id']

'QVFBRDRpUm1HdjRubjV3U01vWEFVcTBqeDY3Z3JuMnlWdTF0QUtsOVVoaFctek9YUm83eENJV2Z6TmUzUlFCTkhaZVpEVDM4b3RuNXRYb092RXdmRFhQYQ=='

In [32]:
len(igs['data']['recent']['sections'])

10

In [35]:
igs['data']['recent']['sections'][0].keys()

dict_keys(['layout_type', 'layout_content', 'feed_type', 'explore_item_info'])

In [67]:
igs['data']['recent']['sections'][1]['layout_content']['medias'][0]['media']['image_versions2']['candidates']

[{'width': 828,
  'height': 828,
  'url': 'https://instagram.fixr3-2.fna.fbcdn.net/v/t51.2885-15/276141787_1149494562481218_4170828401091337593_n.jpg?stp=dst-jpg_e35&_nc_ht=instagram.fixr3-2.fna.fbcdn.net&_nc_cat=106&_nc_ohc=hknT4CF253EAX9M3h4B&edm=ABZsPhsBAAAA&ccb=7-4&ig_cache_key=Mjc5ODYxODcxMjQ1NjI5OTQ2NA%3D%3D.2-ccb7-4&oh=00_AT8EfSUfMG4DY7gozx-Ng148rEXUmZTHPkzeCu3oTN3CzA&oe=623FE621&_nc_sid=4efc9f'},
 {'width': 640,
  'height': 640,
  'url': 'https://instagram.fixr3-2.fna.fbcdn.net/v/t51.2885-15/276141787_1149494562481218_4170828401091337593_n.jpg?stp=dst-jpg_e35_s640x640_sh0.08&_nc_ht=instagram.fixr3-2.fna.fbcdn.net&_nc_cat=106&_nc_ohc=hknT4CF253EAX9M3h4B&edm=ABZsPhsBAAAA&ccb=7-4&ig_cache_key=Mjc5ODYxODcxMjQ1NjI5OTQ2NA%3D%3D.2-ccb7-4&oh=00_AT9wFFPO6zPC8JX5pVDntCr1C8R5_dMCwKRC3fvA3iiO-g&oe=623FE621&_nc_sid=4efc9f'},
 {'width': 480,
  'height': 480,
  'url': 'https://instagram.fixr3-2.fna.fbcdn.net/v/t51.2885-15/276141787_1149494562481218_4170828401091337593_n.jpg?stp=dst-jpg_e35_s4

In [46]:
def find(key, dictionary):
    if isinstance(dictionary, dict):
        for k, v in dictionary.items():
            if k == key:
                yield v
            elif isinstance(v, dict):
                for result in find(key, v):
                    yield result
            elif isinstance(v, list):
                for d in v:
                    for result in find(key, d):
                        yield result
    else:
        if isinstance(dictionary, list):
            for d in dictionary:
                for result in find(key, d):
                    yield result

In [85]:
datas = [a for a in find("next_max_id",igs)]
len(datas)

17

In [86]:
datas

[17938046782901426,
 17905061603110185,
 17904981713045047,
 17888278484321597,
 17915819572754214,
 17859069539569242,
 17930285626602118,
 18186551338121739,
 17953276426466062,
 'c7e36d7ebff543b99b6eea9ea7cbbefa',
 18025086748359541,
 17938153594978615,
 17918307458500625,
 17917313921342784,
 17964835876583921,
 18289119724026991,
 'QVFBVllBekJsYXFBdVJMMmluelJoTEJJQWlLY1ZJY2tLNDZQS3RtcE5GUHdPNWtqSnd0cnpjMDNPZmhGLU51UVluRHhCbndQWXcwYkl1SkhvdlRDNkw3Ng==']

In [72]:
unique_igs = [b['url'] for a in datas for b in a if b['height']==1080]
len(unique_igs)

49

In [87]:
len(set(unique_igs))

len('QVFBVllBekJsYXFBdVJMMmluelJoTEJJQWlLY1ZJY2tLNDZQS3RtcE5GUHdPNWtqSnd0cnpjMDNPZmhGLU51UVluRHhCbndQWXcwYkl1SkhvdlRDNkw3Ng')

118

In [129]:
def get_images(hashtag,scroll_page=3):
    start = 1
    base_url = f'https://i.instagram.com/api/v1/tags/{hashtag}/sections/'
    hashtag_url = f"https://www.instagram.com/explore/tags/{hashtag}/?__a=1"
    str_suff = {'include_persistent': ' 0','max_id': None,
                 'page': '1','surface': ' grid','tab': ' recent'}
#     res = cr.get(hashtag_url,headers=h).json()
    res = cr.get(hashtag_url,request_type='get',parser='json',headers=h)
    print(res)
    images = [a for a in find("candidates",res)]
    unique_igs = list(set([b['url'] for a in images for b in a if b['height']==1080]))
    next_max_id = [b for b in [a for a in find("next_max_id",res)] if len(str(b))>50][0]
    print(next_max_id)
    str_suff['max_id'] = next_max_id
    str_suff['page'] = start
    while scroll_page>0:
        print(str_suff)
        res = cr.get(base_url,request_type='post',parser='json',headers=h,params=str_suff)
        print(res)
        start += 1
        images = [a for a in find("candidates",res)]
        unique_igs += list(set([b['url'] for a in images for b in a if b['height']==1080]))
        next_max_id = [b for b in [a for a in find("next_max_id",res)] if len(str(b))>50][0]
        str_suff['max_id'] = next_max_id
        str_suff['page'] = start
        scroll_page -=1
        time.sleep(random.randint(1,5))
    return unique_igs

In [130]:
igs = get_images("toonapp")

{'graphql': {'hashtag': {'id': '17843708884043911', 'name': 'toonapp', 'allow_following': False, 'is_following': False, 'is_top_media_only': False, 'profile_pic_url': 'https://scontent-nrt1-1.cdninstagram.com/v/t51.2885-15/275794901_1647555402279847_6435222795164722303_n.webp?stp=dst-jpg_e35_s150x150&_nc_ht=scontent-nrt1-1.cdninstagram.com&_nc_cat=107&_nc_ohc=2XBmK8D5l_IAX9ApfLH&edm=ABZsPhsBAAAA&ccb=7-4&oh=00_AT-mw_nLrWO45cqicyBTYB1_fIC4kGp07ACb4ib5DiVwPA&oe=623EF010&_nc_sid=4efc9f', 'edge_hashtag_to_media': {'count': 48389, 'page_info': {'has_next_page': True, 'end_cursor': 'QVFDYTg2RjBQYi16ZUtRdmVUVlJKRUFudjNaSHpPZGZVNUJZa0tfRHYzMHltdGlJTlhOUHJNWjFFNm1sVEdrelBxc1VyOXJ1Si1XbmwtSklzazhpQnBWUA=='}, 'edges': [{'node': {'comments_disabled': False, '__typename': 'GraphImage', 'id': '2798869043575693354', 'edge_media_to_caption': {'edges': [{'node': {'text': '🌐 AGARRA ESTA APP: ToonApp.\nToonApp é um editor de imagens que permite transformar fotos, especialmente selfies, em caricaturas e/ou

IndexError: list index out of range

In [97]:
len(igs)

44

In [98]:
igs

['https://instagram.fixr3-1.fna.fbcdn.net/v/t51.2885-15/221668396_334468371649591_4797703493906332040_n.jpg?stp=dst-jpg_e35&_nc_ht=instagram.fixr3-1.fna.fbcdn.net&_nc_cat=107&_nc_ohc=FsFJXM2dDEAAX-GGQS2&edm=ABZsPhsBAAAA&ccb=7-4&ig_cache_key=MjYyMzU1NjY1OTA4MDEyOTYxNQ%3D%3D.2-ccb7-4&oh=00_AT-IgCqO-iO9tO-Fsq7bhf3N2EtMwtsXeTBS3xxLI27E-g&oe=623FCA1E&_nc_sid=4efc9f',
 'https://instagram.fixr3-1.fna.fbcdn.net/v/t51.2885-15/276015961_1056316651630704_927842678030476203_n.jpg?stp=dst-jpg_e35&_nc_ht=instagram.fixr3-1.fna.fbcdn.net&_nc_cat=100&_nc_ohc=AqI04ZYhSMMAX_BbHAa&edm=ABZsPhsBAAAA&ccb=7-4&ig_cache_key=Mjc5NzkxOTg5OTkyNzE4MDE2NQ%3D%3D.2-ccb7-4&oh=00_AT8Vjy9xxSvXkLeFHI9EYvQi_QbOSfBS94makBKIn8jmvA&oe=624069C6&_nc_sid=4efc9f',
 'https://instagram.fixr3-1.fna.fbcdn.net/v/t51.2885-15/276037007_167897745576935_3760651959860370905_n.webp?stp=dst-jpg_e35&_nc_ht=instagram.fixr3-1.fna.fbcdn.net&_nc_cat=111&_nc_ohc=Op-erIeoIwIAX9Lv02o&edm=ABZsPhsBAAAA&ccb=7-4&ig_cache_key=Mjc5NzQ1ODU1NjgzODA5MTk5Ng%3

In [81]:
dict((a.split(":")[0],a.split(":")[-1]) for a in """include_persistent: 0
max_id: QVFBTlo4Q0NiZ1Y3aWlVV0ZicURxdGhfejRYdGlSOENVZVRZbWZDei1FS3ZlWmtJYzFlckFMY0VxRUxRMUFWQTZvUmx4TmVuenl1SE9haVFkWUdHZ1gyMA==
page: 2
surface: grid
tab: recent""".splitlines())

{'include_persistent': ' 0',
 'max_id': ' QVFBTlo4Q0NiZ1Y3aWlVV0ZicURxdGhfejRYdGlSOENVZVRZbWZDei1FS3ZlWmtJYzFlckFMY0VxRUxRMUFWQTZvUmx4TmVuenl1SE9haVFkWUdHZ1gyMA==',
 'page': ' 2',
 'surface': ' grid',
 'tab': ' recent'}

In [83]:
get_images('toonapp')

{'message': 'CSRF token missing or incorrect', 'status': 'fail'}